In [1]:
import time
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
%load_ext autoreload
%autoreload 2

In [2]:
from intabs_multi.dataset import InnDataSet
d = InnDataSet("iris")
data_name = "iris"

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [3]:
from intabs_multi.classifier_multi import *
num_h_neurons = 5
epochs = 50
cross_validation(d, num_h_neurons, epochs, data_name)

Using cuda device
accuracy: 0.9314285714285713+-0.045235588903001604


In [4]:
torch_model = train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, num_h_neurons, epochs, data_name)
model = InnModel(d, torch_model, num_h_neurons)

Evaluations on training data

               precision    recall  f1-score   support

         (0)      1.000     1.000     1.000        17
         (1)      1.000     0.900     0.947        20
         (2)      0.917     1.000     0.957        22

    accuracy                          0.966        59
   macro avg      0.972     0.967     0.968        59
weighted avg      0.969     0.966     0.966        59

Evaluations on testing data

               precision    recall  f1-score   support

         (0)      1.000     1.000     1.000         4
         (1)      1.000     1.000     1.000         5
         (2)      1.000     1.000     1.000         6

    accuracy                          1.000        15
   macro avg      1.000     1.000     1.000        15
weighted avg      1.000     1.000     1.000        15



In [5]:
rt_models, val_set, val_y_set = get_retrained_models_and_validation_set(d, model)

100%|██████████| 5/5 [00:00<00:00, 24.50it/s]


In [16]:
from intabs_multi.methods_multi import rnce
from intabs_multi.util import eval_empirical_robustness, get_test_inputs
val_delta = 0.015
ces = rnce(val_set, d, model, delta=val_delta, y_target=2)
print(eval_empirical_robustness(rt_models, ces, y_target=2))

53it [00:00, 215.66it/s]

True


In [17]:
# val delta
test_set, test_set_df, test_set_full_df = get_test_inputs(d, model, num_inputs=20)

In [18]:
from intabs_multi.classifier_multi import get_retrained_models_all
rt_models_eval = get_retrained_models_all(d, model, num_h_neurons, epochs)

100%|██████████| 5/5 [00:00<00:00, 11.71it/s]


In [19]:
from intabs_multi.methods_multi import run_all
run_all(d, model, rt_models_eval, val_delta)

53it [00:00, 221.15it/s]
20it [00:01, 10.58it/s]
53it [00:00, 211.10it/s]
20it [00:01, 10.12it/s]
53it [00:00, 230.00it/s]
20it [00:01, 10.66it/s]
53it [00:00, 230.98it/s]
20it [00:01, 11.04it/s]
53it [00:00, 231.50it/s]
20it [00:01, 10.33it/s]

average results
+----------+------------+--------+-------+-------+--------------+
| name     |   coverage |   cost |   lof |   vm2 |   vdelta-val |
+==========+============+========+=======+=======+==============+
| nnce     |          1 |  0.393 | 1.475 | 0.747 |            0 |
| rnce     |          1 |  0.438 | 1.503 | 1     |            1 |
| rnce-opt |          1 |  0.438 | 1.503 | 1     |            1 |
+----------+------------+--------+-------+-------+--------------+
std results
+----------+------------+--------+-------+-------+--------------+
| name     |   coverage |   cost |   lof |   vm2 |   vdelta-val |
+==========+============+========+=======+=======+==============+
| nnce     |          0 |  0.002 | 0.026 | 0.004 |            0 |
| rnce     |          0 |  0.003 | 0.002 | 0     |            0 |
| rnce-opt |          0 |  0.003 | 0.002 | 0     |            0 |
+----------+------------+--------+-------+-------+--------------+
